In [6]:
# !pip list | grep fastapi

In [1]:
import json
import pandas as pd
from uuid import uuid4
from pprint import pprint
from pymongo import MongoClient, DESCENDING
from argparse import ArgumentParser
from config import settings
collection_name = 'blocks'

connection_string = settings.MONGO_CONNECTION_STRING
client = MongoClient(connection_string)
print("Connected to MongoDB successfully!")

Connected to MongoDB successfully!


In [2]:
db = client.archive
docs = db[collection_name].find().sort({'created_at': -1})

In [3]:
df = pd.DataFrame(list(docs)[::-1])
df

,_id,content,created_at,creator,topic,main_thread_id
0,c3b41f3c-692d-4a27-9d98-04b1fa0919d0,Hi <@U066Q9JAU3B>\nNice to see you at our proj...,2024-04-11T07:49:45.714149,UU43NJY8K,Project Kickoff and Preparation,199c34
1,bfa16fbb-2201-4863-abfa-6cc9ec25ce33,"Hi <@UU43NJY8K>,\nThanks for inviting us.",2024-04-11T08:22:09.374109,U066Q9JAU3B,Project Kickoff and Preparation,199c34
2,21e66f4a-fbfc-4bc7-91d6-cc54ec427d57,Hi <@U066Q9JAU3B> <@U06855K24SE>\nDon't you mi...,2024-04-11T08:59:32.370529,UU43NJY8K,Project Kickoff and Preparation,199c34
3,effa84d1-cb39-4ea5-937d-95ea328afe5c,Works for me.,2024-04-11T09:02:28.000959,U066Q9JAU3B,Project Kickoff and Preparation,199c34
4,7e92acd1-4103-49d0-97db-1e4ebd891e69,"Hi Yogesh,\nPlease do not hesitate to share al...",2024-04-12T08:04:20.532329,UU43NJY8K,Project Kickoff and Preparation,199c34
...,...,...,...,...,...,...
245,2348069f-592d-4de9-94ee-136cee9b9907,"No, currently there is no overlap (I need to a...",2024-05-16T07:33:00.645309,U05R0KHPU8N,Going back and forth on the best approach for ...,f0c9ed
246,6363572d-f176-4851-8592-d46b29c4f6ae,"without an overlapping window, we have no way ...",2024-05-16T11:24:58.207359,U066Q9JAU3B,Going back and forth on the best approach for ...,f0c9ed
247,da42b289-45b7-4e06-88e6-7b469a362188,"<@U066Q9JAU3B> Indeed, my recent experiments s...",2024-05-16T11:27:03.332109,U05R0KHPU8N,Going back and forth on the best approach for ...,f0c9ed
248,ce942fb1-0179-4655-8d38-f9f41475af6d,"with the overlap, we can even increase the bat...",2024-05-16T11:30:49.711009,U066Q9JAU3B,Going back and forth on the best approach for ...,f0c9ed


In [4]:
mapping = dict(list(df.groupby(['topic', 'main_thread_id']).groups))
mapping

{'Discussion about Googles Meet link and code sharing': 'cfe87f',
 'Discussion about additional metrics for evaluating similarity of messages': '07807d',
 'Discussion about experimenting with AI models and scheduling a meeting': '6c9f78',
 'Discussion about project results and feedback': '70737b',
 'Discussion on LLM and BERTopic performance': '6e364e',
 'Discussion on Model Deployment and Evaluation': '808935',
 'Discussion on clustering algorithm and features': '714e81',
 'Discussion on models and server options': '9d3e7c',
 'GitHub and MongoDB access': 'e264d1',
 'Go programming and hybrid search': '6946b3',
 'Going back and forth on the best approach for handling machine learning models': 'f0c9ed',
 'GoogMeet Invitation and Discussion on Costs': '3ffd05',
 'Googledocs and model deployment': '22a66b',
 'Greetings and well wishes': '1b3652',
 'Issues with Cloudflare and server deployment': '4b0065',
 'LLM experimentation': '09e0bd',
 'Manual clustering and comparing results': '3b0ebf

In [9]:
# selected_topics_ids = ['ed387b76-1', '948edadf-5', '9f2fb729-3']
selected_topics_ids = ['acc7439c-b', '0a76a334-c']

stage_matching_ids = {'$match': {'main_thread_id': {'$in': selected_topics_ids}}}

stage_grouping = {'$group': {
    '_id': '$main_thread_id',
    'topic': {'$addToSet': '$topic'},
    'topic_messages': {'$push': '$content'}
}}

stage_sorting = {
    "$sort": { "main_thread_id": DESCENDING }
}

pipeline = [
    stage_matching_ids,
    stage_sorting,
    stage_grouping,
]

result = list(db[collection_name].aggregate(pipeline))
pprint(list(result))

[{'_id': 'acc7439c-b',
  'topic': ['Project Kickoff and Preparation'],
  'topic_messages': ['Hi <@U066Q9JAU3B>\n'
                     'Nice to see you at our project channel :tada:\n'
                     'The other members of our team will be added asap after '
                     'our kick-off meeting.',
                     '<@U05B5FXLP62> <@U06RU2JKMDG> Here is the dataset for '
                     'the project',
                     'Hi <@UU43NJY8K>,\nThanks for inviting us.',
                     'Hi <@U066Q9JAU3B> <@U06855K24SE>\n'
                     "Don't you mind if we shift our kick-off meeting 30 "
                     'minutes tomorrow?\n'
                     'For 7:30 am your time please?',
                     'Thank you for your flexibility to get on a call on a '
                     'Friday evening. I lived many years in Europe and I know '
                     'that this is a big deal!',
                     'Here are few files from the folder that I think is a

In [8]:
print(f'{result[0]["topic"][0]}\n\n\n\n', f'\n{"-" * 100}\n'.join(result[0]['topic_messages']))

Going back and forth on the best approach for handling machine learning models



 yeah.. making batch size too small will mean we are not casting big enough net to catch all messages in a conversation.. looks like a second llm may be a better option.
Are your batches overlapping? they should be i think.
----------------------------------------------------------------------------------------------------
No, currently there is no overlap (I need to adjust the backend to allow overlap and handle it correctly).
----------------------------------------------------------------------------------------------------
<@U066Q9JAU3B> Indeed, my recent experiments show this:
batch size 12

overlap
0 -- 17.4%
2 -- 14.6%
4 -- 11.9%
6 -- 4%
8 -- 1.5%
10 -- 0.2%

which I believe supports your statement.
----------------------------------------------------------------------------------------------------
without an overlapping window, we have no way to determine conversation boundaries and we risk forcin